### 1. Linear model


#### Object function 
$min$  $Z = \sum_{j=1}^{M} \sum_{g=1}^{G} \sum_{t=1}^{T} P_{g,j,t}* X_{g,j,t} + \sum_{j=1}^{M} (( 1 - \sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}) * O_j)$

#### Decision Variable
\begin{equation}
  X_{g,j,t} = \left \{
  \begin{aligned}
    &1, && \text{if team g starts working on engine j at day t}\  \\
    &0, && \text{otherwise}
  \end{aligned} \right.
\end{equation} 

#### Constraints 
##### Maintenance of engine j can be performed at most once during the planning period

$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}\leq 1$ for all $j \in M$

##### Maintaince must be completed within the planning period
$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t} * (t + \mu_{g,j} -1) \leq T$ for all $j \in M$

##### Teams can only work on one engine at a time
$\sum_{j=1}^{M} X_{g,j,t_a} * (\sum_{t=t_a}^{min \{ T,t + \mu_{g,j} -1\}} \sum_{j=1}^{M} X_{g,j,t_a}) \leq 1$ for all $g \in G, t_a \in T$